In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import random
import zipfile
import time
import shutil
from sklearn.metrics import log_loss

In [2]:
gender_train = pd.read_csv('gender_age_train.csv')
gender_test = pd.read_csv('gender_age_test.csv')
phone_brand = pd.read_csv('phone_brand_device_model.csv')
app_events = pd.read_csv('app_events.csv', dtype={'app_id': np.str})
app_labels = pd.read_csv('app_labels.csv',dtype={'app_id': np.str})
labels = pd.read_csv('label_categories.csv')
events = pd.read_csv('events_new.csv', index_col=0)
app_text = pd.read_csv('app_text.csv')

In [3]:
#app_cat = pd.merge(app_labels,labels,on='label_id')
#app_cat_un = app_cat.groupby('app_id', as_index=False).agg({'category' : lambda x: ' '.join(x)})
#app_cat_un.shape
#app_ev = pd.merge(app_events, app_cat_un, on='app_id')
#app_ev.shape
#app_text = app_ev.groupby('event_id', as_index = False).agg({'category' : lambda x: ' '.join(x), 'app_id' : lambda x: ' '.join(x)})
#app_text.shape
#app_text.to_csv('app_text.csv',index_label=False)

In [4]:
# data - events and app_text
data = pd.merge(events, app_text, on='event_id')

In [5]:
# drop columns from app_ev (?????)
# where is app_ev (?)
app_ev = app_events.drop(['is_installed', 'is_active'],axis=1)

In [6]:
# app_cnt - count of events for any app
app_cnt = app_ev.groupby('app_id', as_index=False)['event_id'].count()

In [7]:
# data - merge of data and app_events
data = pd.merge(data, app_events, on='event_id')

In [8]:
data.head(2)

,event_id,device_id,timestamp,longitude,latitude,hour,weekday,long_lat,is_morning,is_night,is_day,is_evening,category,app_id_x,app_id_y,is_installed,is_active
0,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,0,6,103.65_30.97,False,True,False,False,Property Industry 1.0 Industry tag Relatives 1...,5927333115845830913 -5720078949152207372 -1633...,5927333115845830913,1,1
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,0,6,103.65_30.97,False,True,False,False,Property Industry 1.0 Industry tag Relatives 1...,5927333115845830913 -5720078949152207372 -1633...,-5720078949152207372,1,0


In [9]:
# TODO drop duplicates from data (!!!)
# duplicates by event_id, device and timestamp

In [10]:
data = data.drop_duplicates(subset=['device_id'])

In [11]:
# kick duplicates from phone_brand
phone_brand.drop_duplicates(subset='device_id',keep='first', inplace=True)

In [12]:
# change bool values to int type
#data.is_morning = data.is_morning.astype(int)
#data.is_night = data.is_night.astype(int)
#data.is_day = data.is_day.astype(int)
#data.is_evening = data.is_evening.astype(int)

In [13]:
# train and test data - merge of gender... and data
train_data = pd.merge(gender_train, data, on='device_id', how='left')
test_data = pd.merge(gender_test, data, on='device_id', how='left')

In [14]:
# train and test data - merge of ..._data and phone_brand
train_data = pd.merge(train_data, phone_brand, on='device_id')
test_data = pd.merge(test_data, phone_brand, on='device_id')

In [15]:
# fill na values with no_data
train_data = train_data.fillna('no_data')
test_data = test_data.fillna('no_data')

In [16]:
# app_events_sum - grouped by event app_events
#app_events_sum = app_events.groupby(['event_id'])

In [17]:
# count of apps, count of installed, count of active
#app_cnt = app_events.groupby(['event_id'], as_index=False)['app_id'].count()
#installed_sum = app_events.groupby(['event_id'], as_index=False)['is_installed'].sum()
#active_sum = app_events.groupby(['event_id'], as_index=False)['is_active'].sum()

In [18]:
# summary of previous counts
#app_data_med = pd.merge(app_cnt, installed_sum, on=['event_id'])
#app_data = pd.merge(app_data_med, active_sum, on=['event_id'])
#app_data['active_share'] = app_data.is_active/app_data.app_id

In [19]:
# drop gender and age from train_data
#train_data = train_data.drop(['gender','age'], axis=1)

In [20]:
group = train_data['group']
train_data = train_data[['device_id', 'category', 'app_id_x', 'phone_brand', 'device_model']]
test_data = test_data[['device_id', 'category', 'app_id_x', 'phone_brand', 'device_model']]

In [21]:
train_data.category = train_data.category.astype(str)
train_data.app_id_x = train_data.app_id_x.astype(str)
train_data.phone_brand = train_data.phone_brand.astype(str)
train_data.device_model = train_data.device_model.astype(str)

In [22]:
test_data.category = test_data.category.astype(str)
test_data.app_id_x = test_data.app_id_x.astype(str)
test_data.phone_brand = test_data.phone_brand.astype(str)
test_data.device_model = test_data.device_model.astype(str)

In [23]:
train_data['text'] = train_data.category + ' ' + train_data.app_id_x + ' ' + \
train_data.phone_brand + ' ' + train_data.device_model

In [24]:
test_data['text'] = test_data.category + ' ' + test_data.app_id_x + ' ' + \
test_data.phone_brand + ' ' + test_data.device_model

In [25]:
#train_data.head(20)

In [26]:
#test_data.head(20)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
print(train_data.shape)
print(test_data.shape)

(74645, 6)
(112071, 6)


In [29]:
total = pd.concat([train_data, test_data])

In [30]:
total.shape

(186716, 6)

In [31]:
# vectorizer = TfidfVectorizer(min_df=2, max_df=0.75, stop_words='english')
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(total.text.values)

In [32]:
print(X[:train_data.shape[0]].shape)
print(X[train_data.shape[0]:].shape)

(74645, 17687)
(112071, 17687)


In [33]:
'''
# count devices in events
hour_cnt = train_data.groupby(['device_id'], as_index=False)['hour'].count().rename(columns={'hour':'device_cnt'})
# hour activities
hour_avg = train_data.groupby(['device_id'], as_index=False)['hour'].mean().rename(columns={'hour':'hour_avg'})
hour_med = train_data.groupby(['device_id'], as_index=False)['hour'].median().rename(columns={'hour':'hour_med'})
hour_min = train_data.groupby(['device_id'], as_index=False)['hour'].min().rename(columns={'hour':'hour_min'})
hour_max = train_data.groupby(['device_id'], as_index=False)['hour'].max().rename(columns={'hour':'hour_max'})
# weekday activities
weekday_avg = train_data.groupby(['device_id'], as_index=False)['weekday'].mean().rename(columns={'weekday':'weekday_avg'})
weekday_med = train_data.groupby(['device_id'], as_index=False)['weekday'].median().rename(columns={'weekday':'weekday_med'})
weekday_min = train_data.groupby(['device_id'], as_index=False)['weekday'].min().rename(columns={'weekday':'weekday_min'})
weekday_max = train_data.groupby(['device_id'], as_index=False)['weekday'].max().rename(columns={'weekday':'weekday_max'})
# part of day activities
morning_sum = train_data.groupby(['device_id'], as_index=False)['is_morning'].sum().rename(columns={'is_morning':'morning_sum'})
morning_avg = train_data.groupby(['device_id'], as_index=False)['is_morning'].mean().rename(columns={'is_morning':'morning_avg'})
day_sum = train_data.groupby(['device_id'], as_index=False)['is_day'].sum().rename(columns={'is_morning':'day_sum'})
day_avg = train_data.groupby(['device_id'], as_index=False)['is_day'].mean().rename(columns={'is_morning':'day_avg'})
night_sum = train_data.groupby(['device_id'], as_index=False)['is_night'].sum().rename(columns={'is_night':'night_sum'})
night_avg = train_data.groupby(['device_id'], as_index=False)['is_night'].mean().rename(columns={'is_night':'night_avg'})
evening_sum = train_data.groupby(['device_id'], as_index=False)['is_evening'].sum().rename(columns={'is_evening':'evening_sum'})
evening_avg = train_data.groupby(['device_id'], as_index=False)['is_evening'].mean().rename(columns={'is_evening':'evening_avg'})
'''

"\n# count devices in events\nhour_cnt = train_data.groupby(['device_id'], as_index=False)['hour'].count().rename(columns={'hour':'device_cnt'})\n# hour activities\nhour_avg = train_data.groupby(['device_id'], as_index=False)['hour'].mean().rename(columns={'hour':'hour_avg'})\nhour_med = train_data.groupby(['device_id'], as_index=False)['hour'].median().rename(columns={'hour':'hour_med'})\nhour_min = train_data.groupby(['device_id'], as_index=False)['hour'].min().rename(columns={'hour':'hour_min'})\nhour_max = train_data.groupby(['device_id'], as_index=False)['hour'].max().rename(columns={'hour':'hour_max'})\n# weekday activities\nweekday_avg = train_data.groupby(['device_id'], as_index=False)['weekday'].mean().rename(columns={'weekday':'weekday_avg'})\nweekday_med = train_data.groupby(['device_id'], as_index=False)['weekday'].median().rename(columns={'weekday':'weekday_med'})\nweekday_min = train_data.groupby(['device_id'], as_index=False)['weekday'].min().rename(columns={'weekday':'w

In [34]:
'''
# count devices in events
hour_cnt_test = test_data.groupby(['device_id'], as_index=False)['hour'].count().rename(columns={'hour':'device_cnt'})
# hour activities
hour_avg_test = test_data.groupby(['device_id'], as_index=False)['hour'].mean().rename(columns={'hour':'hour_avg'})
hour_med_test = test_data.groupby(['device_id'], as_index=False)['hour'].median().rename(columns={'hour':'hour_med'})
hour_min_test = test_data.groupby(['device_id'], as_index=False)['hour'].min().rename(columns={'hour':'hour_min'})
hour_max_test = test_data.groupby(['device_id'], as_index=False)['hour'].max().rename(columns={'hour':'hour_max'})
# weekday activities
weekday_avg_test = test_data.groupby(['device_id'], as_index=False)['weekday'].mean().rename(columns={'weekday':'weekday_avg'})
weekday_med_test = test_data.groupby(['device_id'], as_index=False)['weekday'].median().rename(columns={'weekday':'weekday_med'})
weekday_min_test = test_data.groupby(['device_id'], as_index=False)['weekday'].min().rename(columns={'weekday':'weekday_min'})
weekday_max_test = test_data.groupby(['device_id'], as_index=False)['weekday'].max().rename(columns={'weekday':'weekday_max'})
# part of day activities
morning_sum_test = test_data.groupby(['device_id'], as_index=False)['is_morning'].sum().rename(columns={'is_morning':'morning_sum'})
morning_avg_test = test_data.groupby(['device_id'], as_index=False)['is_morning'].mean().rename(columns={'is_morning':'morning_avg'})
day_sum_test = test_data.groupby(['device_id'], as_index=False)['is_day'].sum().rename(columns={'is_morning':'day_sum'})
day_avg_test = test_data.groupby(['device_id'], as_index=False)['is_day'].mean().rename(columns={'is_morning':'day_avg'})
night_sum_test = test_data.groupby(['device_id'], as_index=False)['is_night'].sum().rename(columns={'is_night':'night_sum'})
night_avg_test = test_data.groupby(['device_id'], as_index=False)['is_night'].mean().rename(columns={'is_night':'night_avg'})
evening_sum_test = test_data.groupby(['device_id'], as_index=False)['is_evening'].sum().rename(columns={'is_evening':'evening_sum'})
evening_avg_test = test_data.groupby(['device_id'], as_index=False)['is_evening'].mean().rename(columns={'is_evening':'evening_avg'})
'''

"\n# count devices in events\nhour_cnt_test = test_data.groupby(['device_id'], as_index=False)['hour'].count().rename(columns={'hour':'device_cnt'})\n# hour activities\nhour_avg_test = test_data.groupby(['device_id'], as_index=False)['hour'].mean().rename(columns={'hour':'hour_avg'})\nhour_med_test = test_data.groupby(['device_id'], as_index=False)['hour'].median().rename(columns={'hour':'hour_med'})\nhour_min_test = test_data.groupby(['device_id'], as_index=False)['hour'].min().rename(columns={'hour':'hour_min'})\nhour_max_test = test_data.groupby(['device_id'], as_index=False)['hour'].max().rename(columns={'hour':'hour_max'})\n# weekday activities\nweekday_avg_test = test_data.groupby(['device_id'], as_index=False)['weekday'].mean().rename(columns={'weekday':'weekday_avg'})\nweekday_med_test = test_data.groupby(['device_id'], as_index=False)['weekday'].median().rename(columns={'weekday':'weekday_med'})\nweekday_min_test = test_data.groupby(['device_id'], as_index=False)['weekday'].mi

In [35]:
'''
result_train = pd.merge(hour_cnt, hour_avg, on='device_id')
result_train = pd.merge(result_train, hour_med, on='device_id')
result_train = pd.merge(result_train, hour_min, on='device_id')
result_train = pd.merge(result_train, hour_max, on='device_id')
result_train = pd.merge(result_train, weekday_avg, on='device_id')
result_train = pd.merge(result_train, weekday_med, on='device_id')
result_train = pd.merge(result_train, weekday_min, on='device_id')
result_train = pd.merge(result_train, weekday_max, on='device_id')
result_train = pd.merge(result_train, morning_sum, on='device_id')
result_train = pd.merge(result_train, morning_avg, on='device_id')
result_train = pd.merge(result_train, day_sum, on='device_id')
result_train = pd.merge(result_train, day_avg, on='device_id')
result_train = pd.merge(result_train, evening_sum, on='device_id')
result_train = pd.merge(result_train, evening_avg, on='device_id')
result_train = pd.merge(result_train, night_sum, on='device_id')
result_train = pd.merge(result_train, night_avg, on='device_id')
'''

"\nresult_train = pd.merge(hour_cnt, hour_avg, on='device_id')\nresult_train = pd.merge(result_train, hour_med, on='device_id')\nresult_train = pd.merge(result_train, hour_min, on='device_id')\nresult_train = pd.merge(result_train, hour_max, on='device_id')\nresult_train = pd.merge(result_train, weekday_avg, on='device_id')\nresult_train = pd.merge(result_train, weekday_med, on='device_id')\nresult_train = pd.merge(result_train, weekday_min, on='device_id')\nresult_train = pd.merge(result_train, weekday_max, on='device_id')\nresult_train = pd.merge(result_train, morning_sum, on='device_id')\nresult_train = pd.merge(result_train, morning_avg, on='device_id')\nresult_train = pd.merge(result_train, day_sum, on='device_id')\nresult_train = pd.merge(result_train, day_avg, on='device_id')\nresult_train = pd.merge(result_train, evening_sum, on='device_id')\nresult_train = pd.merge(result_train, evening_avg, on='device_id')\nresult_train = pd.merge(result_train, night_sum, on='device_id')\nres

In [36]:
'''
result_test = pd.merge(hour_cnt_test, hour_avg_test, on='device_id')
result_test = pd.merge(result_test, hour_med_test, on='device_id')
result_test = pd.merge(result_test, hour_min_test, on='device_id')
result_test = pd.merge(result_test, hour_max_test, on='device_id')
result_test = pd.merge(result_test, weekday_avg_test, on='device_id')
result_test = pd.merge(result_test, weekday_med_test, on='device_id')
result_test = pd.merge(result_test, weekday_min_test, on='device_id')
result_test = pd.merge(result_test, weekday_max_test, on='device_id')
result_test = pd.merge(result_test, morning_sum_test, on='device_id')
result_test = pd.merge(result_test, morning_avg_test, on='device_id')
result_test = pd.merge(result_test, day_sum_test, on='device_id')
result_test = pd.merge(result_test, day_avg_test, on='device_id')
result_test = pd.merge(result_test, evening_sum_test, on='device_id')
result_test = pd.merge(result_test, evening_avg_test, on='device_id')
result_test = pd.merge(result_test, night_sum_test, on='device_id')
result_test = pd.merge(result_test, night_avg_test, on='device_id')
'''

"\nresult_test = pd.merge(hour_cnt_test, hour_avg_test, on='device_id')\nresult_test = pd.merge(result_test, hour_med_test, on='device_id')\nresult_test = pd.merge(result_test, hour_min_test, on='device_id')\nresult_test = pd.merge(result_test, hour_max_test, on='device_id')\nresult_test = pd.merge(result_test, weekday_avg_test, on='device_id')\nresult_test = pd.merge(result_test, weekday_med_test, on='device_id')\nresult_test = pd.merge(result_test, weekday_min_test, on='device_id')\nresult_test = pd.merge(result_test, weekday_max_test, on='device_id')\nresult_test = pd.merge(result_test, morning_sum_test, on='device_id')\nresult_test = pd.merge(result_test, morning_avg_test, on='device_id')\nresult_test = pd.merge(result_test, day_sum_test, on='device_id')\nresult_test = pd.merge(result_test, day_avg_test, on='device_id')\nresult_test = pd.merge(result_test, evening_sum_test, on='device_id')\nresult_test = pd.merge(result_test, evening_avg_test, on='device_id')\nresult_test = pd.merg

In [37]:
#result_train.head()

In [38]:
#result_test.head()

In [39]:
def map_column(table):
    labels = sorted(table.unique())
    mappings = dict()
    for i in range(len(labels)):
        mappings[labels[i]] = i
    table = table.map(mappings)
    return table

In [40]:
#phone_brand['phone+brand'] = phone_brand.phone_brand + phone_brand.device_model
#phone_brand = phone_brand.drop(['phone_brand','device_model'],axis=1)
#phone_brand = pd.get_dummies(phone_brand)

In [41]:
#result_test = pd.merge(result_test, phone_brand, on='device_id')
#result_train = pd.merge(result_train, phone_brand, on='device_id')

In [42]:
#result_train = pd.merge(result_train, gender_train, on='device_id')
#result_train = result_train.drop(['gender','age'],axis=1)

In [43]:
#result_train = map_column(result_train, 'group')

In [44]:
random.seed(2016)

def run_xgb(train, test, target, eta=0.1, random_state=0):
    #eta = 0.1
    max_depth = 3
    subsample = 0.7
    colsample_bytree = 0.7
    start_time = time.time()

    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    params = {
        "objective": "multi:softprob",
        "num_class": 12,
        "booster" : "gbtree",
        "eval_metric": "mlogloss",
        "eta": eta,
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": random_state,
    }
    num_boost_round = 20000
    early_stopping_rounds = 100
    test_size = 0.3

    X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=test_size, random_state=random_state)
    # TODO change split 
    print('Length train:', X_train.shape[0])
    print('Length valid:', X_valid.shape[0])
    #y_train = X_train[target]
    #y_valid = X_valid[target]
    # TODO delete upper code
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    # sparse matrix ???

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

    print("Validating...")
    check = gbm.predict(xgb.DMatrix(X_valid), ntree_limit=gbm.best_iteration)
    score = log_loss(y_valid.tolist(), check)

    print("Predict test set...")
    test_prediction = gbm.predict(xgb.DMatrix(test), ntree_limit=gbm.best_iteration)

    print('Training time: {} minutes'.format(round((time.time() - start_time)/60, 2)))
    return test_prediction.tolist(), score

In [45]:
def create_submission(score, test, prediction):
    # Make Submission
    now = datetime.datetime.now()
    sub_file = 'submission_' + str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print('Writing submission: ', sub_file)
    f = open(sub_file, 'w')
    f.write('device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+\n')
    total = 0
    test_val = test['device_id'].values
    for i in range(len(test_val)):
        str1 = str(test_val[i])
        for j in range(12):
            str1 += ',' + str(prediction[i][j])
        str1 += '\n'
        total += 1
        f.write(str1)
    f.close()

In [46]:
map_group = map_column(group)

In [ ]:
res, score = run_xgb(X[:train_data.shape[0]], X[train_data.shape[0]:], map_group, eta=0.02, random_state=0)
print score
#create_submission(score, result_test, res)

XGBoost params. ETA: 0.02, MAX_DEPTH: 3, SUBSAMPLE: 0.7, COLSAMPLE_BY_TREE: 0.7
('Length train:', 52251)
('Length valid:', 22394)


Will train until eval error hasn't decreased in 100 rounds.
[0]	train-mlogloss:2.482272	eval-mlogloss:2.482627
[1]	train-mlogloss:2.479687	eval-mlogloss:2.480392
[2]	train-mlogloss:2.477185	eval-mlogloss:2.478235
[3]	train-mlogloss:2.474742	eval-mlogloss:2.476124
[4]	train-mlogloss:2.472318	eval-mlogloss:2.474055
[5]	train-mlogloss:2.469983	eval-mlogloss:2.472003
[6]	train-mlogloss:2.467720	eval-mlogloss:2.470004
[7]	train-mlogloss:2.465531	eval-mlogloss:2.468110
[8]	train-mlogloss:2.463319	eval-mlogloss:2.466211
[9]	train-mlogloss:2.461102	eval-mlogloss:2.464329
[10]	train-mlogloss:2.458940	eval-mlogloss:2.462511
[11]	train-mlogloss:2.456899	eval-mlogloss:2.460775
[12]	train-mlogloss:2.454848	eval-mlogloss:2.458979
[13]	train-mlogloss:2.452852	eval-mlogloss:2.457292
[14]	train-mlogloss:2.450871	eval-mlogloss:2.455580
[15]	train-mlogloss:2.448924	eval-mlogloss:2.453948
[16]	train-mlogloss:2.447054	eval-mlogloss:2.452371
[17]	train-mlogloss:2.445212	eval-mlogloss:2.450815
[18]	train-mlo

Validating...
Predict test set...
Training time: 27.71 minutes
2.31697974711


In [48]:
create_submission(score, gender_test, res)

('Writing submission: ', 'submission_2.31697974711_2016-08-16-19-33.csv')


In [47]:
#from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression
import sklearn.cross_validation #импортируем кросс-валидацию

In [48]:
#sc = StandardScaler()
#scaled_train = sc.fit_transform(result_train[result_train.columns.difference(['group','device_id'])])
#scaled_test = sc.fit_transform(result_test[result_test.columns.difference(['group'])])

In [49]:
#import time
#import datetime

#bag_score = []#список результатов
#c_est = [10**-2,10**-1,1,10,100,1000]#значения C для перебора
#lr_models = []
#for c in c_est:
#    start_time = datetime.datetime.now()
#    logr = LogisticRegression(penalty='l2',C=c,tol=0.000001,verbose=True)#логистическая регрессия
#    #lr_models.append(logr.fit(scaled_train_bag, features_target))#обучаем класификатор
#    bag_score.append(sklearn.cross_validation.cross_val_score(logr, scaled_train, result_train['group'].values,cv=3, scoring='log_loss').mean())
#    #к списку результатов добавляем средний получившийся на кросс-валидации результат по roc_auc score
#    print bag_score
#    print 'Time elapsed for c =', c,':', datetime.datetime.now() - start_time

In [ ]:
#print bag_score